## Initial imports

In [9]:

import gym
import torch
import random
from collections import defaultdict, deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque

import numpy as np
import copy
from pprint import pprint
import math
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
import gc
import gym_joinemio
from gym_joinemio.envs.player import RandomPlayer

SyntaxError: invalid syntax (<ipython-input-9-ecd6b6545aee>, line 20)

In [3]:
env = gym.make('joinemio-v0')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Replay memory

In [5]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size) # TODO: Batch size delete

    def __len__(self):
        return len(self.memory)

## Neural Net architecture:

In [6]:
class DQN(nn.Module):
    def __init__(self, h, w, outputs):    # TODO: Change params here. Different input, outputs - action space size
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, sEtride=2)
        self.bn3 = nn.BatchNorm2d(32)

        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

## Training: 
Random vs Random. Deep Q-Learning. Params:

- `n_episodes` (int): maximum number of training epsiodes
- `max_t` (int): maximum number of timesteps per episode _// Not used, because these episodes don't take too long and we like when game's are finished_
- `eps_start` (float): starting value of epsilon, for epsilon-greedy action selection
- `eps_end` (float): minimum value of epsilon 
- `eps_decay` (float): mutiplicative factor (per episode) for decreasing epsilon

In [7]:
BUFFER_SIZE = 200
EPISODES = 10
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 0.996

In [8]:
agent1 = RandomPlayer() # Player1
agent2 = RandomPlayer() # Player2

def dqn(n_episodes= EPISODES, eps_start=EPS_START, eps_end = EPS_END, eps_decay=EPS_DECAY):
    memory_buffer = ReplayMemory(BUFFER_SIZE)
    
    scores = []                         # list containing score from each episode
    scores_window = deque(maxlen=100)   # last 100 scores
    eps = eps_start

    for i_episode in range(1, n_episodes+1):
        print(f"=== EPISODE {i_episode} ===")
        state = env.reset()
        score = 0
        
        # # OPT A:
        # one_game = env.play_one_game(agent1, agent2, each_step_render=False)
        # for e in env.recording: print(e)

        # OPT B:
        players = [agent1, agent2]
        env.observation_space = env.reset()
        action = None
        
        while not env.game.game_state == gym_joinemio.envs.board.GameState.finished:
            current_player = env.game.current_player - 1
            action = players[current_player].get_action(env.observation_space)
            state1 = env.game.board.grid
            env.observation_space, reward, done, info = env.step(action)
            state2 = env.game.board.grid

            # Recording data:
            if (current_player == 1): # (?) Do we do it for both players?
                memory_buffer.push(state1, action, state2, reward.value) 

            score +=  reward.value
            scores.append(score)
            scores_window.append(score)
            env.recording.append((current_player + 1, action, env.rewarder()))
            
            # Analyzing scores
            eps = max(eps*eps_decay,eps_end)
            print('Episode {}\tAverage Score {:.2f}\n'.format(i_episode,np.mean(scores_window)), end="")
            if np.mean(scores_window)>=200.0:
                print('\nEnvironment solve in {:d} epsiodes!\tAverage score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
                torch.save(agent.qnetwork_local.state_dict(),'checkpoint.pth')
                break
        
        print(f"--------\nWinner: {env.game.winner}\n")
        # return env.observation_space, reward, done, info  # reward for player1
    return scores

scores= dqn()

=== EPISODE 1 ===
Episode 1	Average Score 0.02
Episode 1	Average Score 0.04
Episode 1	Average Score 0.05
Episode 1	Average Score 0.06
Episode 1	Average Score 0.07
Episode 1	Average Score 0.08
Episode 1	Average Score 0.10
Episode 1	Average Score 0.11
Episode 1	Average Score 0.12
Episode 1	Average Score 0.13
Episode 1	Average Score 0.14
Episode 1	Average Score 0.15
Episode 1	Average Score 0.17
Episode 1	Average Score 0.18
Episode 1	Average Score 0.19
Episode 1	Average Score 0.20
Episode 1	Average Score 0.27
--------
Winner: 1

=== EPISODE 2 ===
Episode 2	Average Score 0.26
Episode 2	Average Score 0.25
Episode 2	Average Score 0.24
Episode 2	Average Score 0.23
Episode 2	Average Score 0.23
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.22
Episode 2	Average Score 0.23
Episode 2	Average Score 0.23
Episode 2	Average Score 0.23
Episode 2	Averag

Evaluation